In [29]:
# scan every frame for possible matches for the seletcted emotions

# "i don't know" >> 4>3, 7>6 but 4 and 7 are lower than 15 or 16. also 3 and 6 lower than 2 and 5
# "happy/celebrating" >> 4, 3, 6, 7 are higher on y than 1
# "thinking" >> 4 or 7 is between 15 and 1. false if both.

#if emotion is interrupted for more than 10 frames - it is counted as 2 separate instances

In [83]:
def estimate_emotion (csv_file):
    
    import pandas as pd
    
    interruption_countdown_thinking = -1
    interruption_countdown_happy = -1
    interruption_countdown_idk = -1

    thinking_counter = 0
    thinking_frames = 0

    happy_counter = 0
    happy_frames = 0

    idk_counter = 0
    idk_frames = 0
    
    #buffer to get rid of false positives
    idk_frames_buffer = 0
    happy_frames_buffer = 0
    thinking_frames_buffer = 0

    something_else_frames = 0 #unrecognised emotion
    
    data = pd.read_csv(csv_file)

    #iterate over each frame, looking for an emotion
    for index, row in data.iterrows():

        
                                            ######### IDK ###########

        
        # check for idk - hands are higher than elbows, and elbows are lower than shoulders, and hands are wider than elbows
        if row[14] < row[11] and row[23] < row[20] and row[2] < row[11] and row[2] < row[23] and row[13] < row[10] and row[22] > row[19]:
            idk_frames_buffer += 1
            interruption_countdown_idk = 10

            #if the video ends with the next frame, increase emotion occurence counter
            if index == data.shape[0]-1:
                idk_frames += idk_frames_buffer
                idk_counter += 1
            continue
            
        if interruption_countdown_idk > -1:
            # if countdown is still on    
            if interruption_countdown_idk > 0 and index != data.shape[0]-1:
                interruption_countdown_idk -= 1
                idk_frames += 1
                continue

            # if it has been more than 10 frames since emotion was last detected ot it is the end of the video, increase emotion counter
            if interruption_countdown_idk == 0 or index == data.shape[0]-1:
                if idk_frames_buffer > 15:
                    idk_frames += idk_frames_buffer
                    idk_counter += 1
                else:
                    something_else_frames += idk_frames_buffer
                something_else_frames += 1
                idk_frames_buffer = 0
                interruption_countdown_idk = -1


                                            ######### HAPPY ###########

        
        # check for happy/celebrating emotion - hand and elbows are higher than shoulders 
        if row[14] < row[5] and row[23] < row[5] and row[20] < row[5] and row[11] < row[5]:
            happy_frames_buffer += 1
            interruption_countdown_happy = 10

            #if the video ends with the next frame, increase emotion occurence counter
            if index == data.shape[0]-1:
                happy_frames += happy_frames_buffer
                happy_counter += 1
            continue
            
        if interruption_countdown_happy > -1:
            # if countdown is still on    
            if interruption_countdown_happy > 0 and index != data.shape[0]-1:
                interruption_countdown_happy -= 1
                happy_frames += 1
                continue

            # if it has been more than 10 frames since emotion was last detected ot it is the end of the video, increase emotion counter
            if interruption_countdown_happy == 0 or index == data.shape[0]-1:
                if happy_frames_buffer > 15:
                    happy_frames += happy_frames_buffer
                    happy_counter += 1
                else:
                    something_else_frames += happy_frames_buffer
                something_else_frames += 1
                happy_frames_buffer = 0
                interruption_countdown_happy = -1

            
                                            ######### THINKING ###########

        
        # check for thinking emotion - if one of hands is higher than both elbows but lower than nose and is betweeen shoulders
        if (row[14] < row[11] and row[14] < row[20] and row[14] > row[1] and row[13] > row[7] and row[13] < row[16] and row[23] > row[5] ) or (row[23] < row[11] and row[23] < row[20] and row[23] > row[1] and row[22] > row[7] and row[22] < row[16] and row[14] > row[5]):
            thinking_frames_buffer += 1
            interruption_countdown_thinking = 10

            #if the video ends with the next frame, increase emotion occurence counter
            if index == data.shape[0]-1:
                thinking_frames += thinking_frames_buffer
                thinking_counter += 1
            continue

        if interruption_countdown_thinking > -1:
            # if countdown is still on    
            if interruption_countdown_thinking > 0 and index != data.shape[0]-1:
                interruption_countdown_thinking -= 1
                thinking_frames += 1
                continue

            # if it has been more than 10 frames since emotion was last detected ot it is the end of the video, increase emotion counter
            if interruption_countdown_thinking == 0 or index == data.shape[0]-1:
                if thinking_frames_buffer > 5:
                    thinking_frames += thinking_frames_buffer
                    thinking_counter += 1
                else:
                    something_else_frames += thinking_frames_buffer
                something_else_frames += 1
                thinking_frames_buffer = 0
                interruption_countdown_thinking = -1

        # if nothing is detected - it is garbage
        else:
            something_else_frames += 1
    
    # remove false positives for idk pose when lifting hand for "happy"
    if idk_frames < 15:
        something_else_frames += idk_frames
        idk_frames = 0
    
    print("idk frames:", idk_frames, "idk time:", idk_frames/30.,"idk counter:",idk_counter)
    print("happy frames:", happy_frames, "happy time:", happy_frames/30.,"happy counter:",happy_counter)
    print("thinking frames:", thinking_frames, "thinking time:", thinking_frames/30.,"thinking counter:",thinking_counter)
    print("something else frames:", something_else_frames, "something else time:", something_else_frames/30.)



In [87]:
estimate_emotion("test2.csv")

idk frames: 185 idk time: 6.166666666666667 idk counter: 2
happy frames: 175 happy time: 5.833333333333333 happy counter: 2
thinking frames: 202 thinking time: 6.733333333333333 thinking counter: 3
something else frames: 178 something else time: 5.933333333333334
